## Data Mining project: Discover and describe areas of interest an events from geo-located data

## Task #1: Import Dataset and Librarie

## Library Installation

In [ ]:
# installation of required libraries and dependencies
# numeric calculations
! pip install numpy==1.26.0 
# data frames 
! pip install pandas==2.1.1 
# machine learning algorithms 
! pip install scikit-learn==1.5.1 
! pip install scipy==1.12.0
# plotting 
! pip install plotly==5.24.1 
! pip install matplotlib==3.8.0 
! pip install seaborn==0.13.2 
! pip install plotly-express==0.4.1 
! pip install chart-studio==1.1.0 
# web app library 
! pip install streamlit==1.37.1 
# association rules
! pip install mlxtend==0.23.3 
! pip install folium==0.19.4
! pip install geopy==2.4.1
! pip install pyproj

## Import

In [129]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.colors as mcolors
from geopy.distance import geodesic
from scipy.spatial import ConvexHull, QhullError

from sklearn.cluster import AgglomerativeClustering,  DBSCAN  ,linkage_tree  
from scipy.cluster.hierarchy import dendrogram
import folium
import seaborn as sns


# Data cleaning and preparation

In [ ]:
# Charger le fichier CSV
data = pd.read_csv("../data/dataSet.csv", sep=",", low_memory=False)

data.info()

### Type Conversion

In [ ]:
# Sélectionner les colonnes 11 et 12 (indices 10 et 11 en pandas)
columns_to_check = data.iloc[:, [11, 12]]

# Parcourir chaque colonne sélectionnée
for col in columns_to_check.columns:
    print(f"Analyse de la colonne: {col}")
    
    # Trouver les types de données présents
    types_present = data[col].map(type).value_counts()
    print("Types présents dans cette colonne :")
    print(types_present)
    
    # Afficher l'utilisation de la mémoire pour cette colonne
    memory_usage = data[col].memory_usage(deep=True)
    print(f"Utilisation de la mémoire pour cette colonne : {memory_usage} bytes\n")

for col in columns_to_check.columns:
    print(f"Conversion de la colonne: {col}")
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)  # Convertir en float, remplacer les erreurs par NaN

    # Vérifier le résultat
    print(f"Colonne {col} après conversion:")
    print(data[col].head())  # Afficher les premières lignes pour valider
    print(f"Types après conversion: {data[col].map(type).value_counts()}\n")

In [ ]:
#rename columms
data.columns = data.columns.str.strip()
print("Colonnes après nettoyage :", data.columns.tolist())

### NULL Values

In [ ]:
#Find null values
data.isna().sum()

In [ ]:
print(f"Initial: {len(data)}")
print(data.columns)
columns_to_clean = [
					'lat', 'long', 'date_taken_minute', 'date_taken_hour', 
					'date_taken_day', 'date_taken_month', 'date_taken_year', 
					'date_upload_minute', 'date_upload_hour', 'date_upload_day', 
					'date_upload_month', 'date_upload_year'
					]
data_cleaned = data.dropna(subset=columns_to_clean)
print(f"After removing missing values: {len(data_cleaned)}")

### Duplicates

In [ ]:
# Check if there are any duplicates
data_cleaned.duplicated().sum()

In [ ]:
# Remove duplicates if necessary
print(f"Initial: {len(data_cleaned)}")
data_cleaned = data_cleaned.drop_duplicates(subset=['user','lat','long','title','date_taken_year','date_taken_month','date_taken_day'],keep='first')
print(f"After removing duplicates: {len(data_cleaned)}") 

### Columns

Note that the data with a non-null values at column 16 is sorted differently. We conclude that it is hard to determine which value belongs to which column since date_taken_minute, date_taken_month can be ambigous.

In [ ]:
columns = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']
column_data = data_cleaned.dropna(subset=columns, how='all')

print(column_data.iloc[0])

The rows that have non-null values in columns 16, 17 and 18 are retracted from the dataset. These columns thus serve no purpose and are removed as well.

In [ ]:
# Specify the columns to check for non-null values
columns_to_check = ['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']

# Drop rows where any of the specified columns are not null
print(f"Initial: {len(data_cleaned)}")
data_cleaned = data_cleaned[data_cleaned[columns_to_check].isnull().all(axis=1)]
print(f"After removing non-null values: {len(data_cleaned)}")

# Drop the columns
data_cleaned = data_cleaned.drop(columns=columns)

print(data_cleaned.columns)

### Save Cleaned Data

In [139]:
data_cleaned.to_csv("../data/data-cleaned.csv", index=False)

## Descriptive Statistics

In [ ]:
data_cleaned.describe(exclude=[object])

## Data Visualisation

### Geolocalisation Marker

In [ ]:
m = folium.Map(location=(45.764, 4.8357), zoom_start=12)

# Sample a subset of the data (e.g., 500 points)
sampled_data = data_cleaned.sample(n=1000, random_state=1)

latitude_col = sampled_data['lat']
longitude_col = sampled_data['long']

for i in range(0, len(sampled_data)):
	folium.Marker([latitude_col.iloc[i], longitude_col.iloc[i]]).add_to(m)

m

### Geolocalisation Heatmap

In [ ]:
from folium.plugins import HeatMap

m = folium.Map(location=(45.764, 4.8357), zoom_start=12)

# Sample a subset of the data (e.g., 500 points)
sampled_data = data_cleaned.sample(n=1000, random_state=1)

latitude_col = sampled_data['lat']
longitude_col = sampled_data['long']

heat_data = [[row['lat'], row['long']] for index, row in sampled_data.iterrows()]
HeatMap(heat_data).add_to(m)

m

## Clustering

### K-Means

The only data necessary to create clusters here would be the latitude and longitude. We'll drop all other columns as it is not necessary.

In [ ]:
# Specify the columns to keep
columns = ['lat', 'long']

# Drop all columns except the specified ones
df_clustering = data_cleaned[columns]
print(df_clustering.head())

Although the longitude and latitude have comparable scales, we can use the StandardScaler to normalize the data

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering)
scaled_data_df = pd.DataFrame(data=scaled_data, columns=df_clustering.columns)

plt.figure(figsize=(10, 6))
plt.scatter(scaled_data_df['lat'], scaled_data_df['long'], alpha=0.5)
plt.title('Scaled Latitude and Longitude')
plt.xlabel('Scaled Latitude')
plt.ylabel('Scaled Longitude')
plt.grid(True)
plt.show()

Interestingly, the same pattern observed in the heatmap is reproduced with the scaled data. This is expected, as the data has been proportionally scaled

We'll find the optimal number of clusters using Elbow method

In [ ]:
# Range of k values to try
k_values = range(1, 40)
sum_square = []

for k in k_values:
	kmeans = KMeans(n_clusters=k, init='k-means++')
	kmeans.fit(scaled_data_df)
	sum_square.append(kmeans.inertia_)

plt.plot(k_values, sum_square)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum square')
plt.show()


The optimal number of clusters can be interpreted to be between 6-10. We found that with 6-10 clusters, the granularity of places is not as well defined as the number of points are extremely vast and spreaded.

In [ ]:
kmeans = KMeans(n_clusters=70, init='k-means++')
kmeans.fit(scaled_data_df)
data_cleaned['cluster_kmeans'] = kmeans.labels_ 
print(data_cleaned[['cluster_kmeans', 'lat', 'long']])

In [ ]:
# Center map
m = folium.Map(location=[data_cleaned['lat'].mean(), data_cleaned['long'].mean()], zoom_start=12)

palette = sns.color_palette("hsv", n_colors=data_cleaned['cluster_kmeans'].nunique())
colors = [mcolors.rgb2hex(color) for color in palette]

centroids = scaler.inverse_transform(kmeans.cluster_centers_)
sampled_data = data_cleaned.sample(n=20000, random_state=1)
cluster_counts = sampled_data['cluster_kmeans'].value_counts()

valid_clusters = cluster_counts[cluster_counts >= 3].index.tolist()

filtered_sampled_data = sampled_data[sampled_data['cluster_kmeans'].isin(valid_clusters)]
cluster_id_sample = filtered_sampled_data['cluster_kmeans'].unique().tolist()

print(cluster_id_sample) 

# Filter points based on distance to centroids
filtered_points = filtered_sampled_data[filtered_sampled_data.apply(
    lambda row: any(geodesic((row['lat'], row['long']), centroid).m <= 300 for centroid in centroids), axis=1)]

# Map markers
for i, row in filtered_points.iterrows():
    point = (row['lat'], row['long'])
    folium.CircleMarker(
        location=point,
        radius=1,
        color=colors[row['cluster_kmeans']],
        fill=True,
        fill_color=colors[row['cluster_kmeans']]
    ).add_to(m)

# Centroids and hulls
for cluster_id in cluster_id_sample:
    cluster_points = filtered_points[filtered_points['cluster_kmeans'] == cluster_id][['lat', 'long']].values
    if len(cluster_points) >= 3:  # ConvexHull requires at least 3 points
        try: 
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Skipping cluster {cluster_id} due to QhullError")
    centroid = centroids[cluster_id]
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Centroid {cluster_id}',
    ).add_to(m)

m.save("../html/kmeans_map.html")

### H-Clust

In [ ]:
df_clustering_without_duplicates = df_clustering.drop_duplicates() # Drop duplicates
print(f"Initial: {len(df_clustering)}")
print(f"After removing duplicates: {len(df_clustering_without_duplicates)}")
df_clustering_without_duplicates = df_clustering_without_duplicates.sample(n=10000, random_state=1) # Sample a subset of the data
scaler = StandardScaler()
scaled_data_duplicates = scaler.fit_transform(df_clustering_without_duplicates)
scaled_data_duplicates_df = pd.DataFrame(data=scaled_data_duplicates, columns=df_clustering.columns)

plt.figure(figsize=(10, 6))
plt.scatter(scaled_data_duplicates_df['lat'], scaled_data_duplicates_df['long'], alpha=0.5)
plt.title('Scaled Latitude and Longitude')
plt.xlabel('Scaled Latitude')
plt.ylabel('Scaled Longitude')
plt.grid(True)
plt.show()


In [149]:
# Vérifier que 'lat' et 'long' sont bien présentes dans le DataFrame
if 'lat' not in df_clustering_without_duplicates.columns or 'long' not in df_clustering_without_duplicates.columns:
    raise ValueError("Les colonnes 'lat' et 'long' doivent être présentes dans le DataFrame.")

# Si 'hierarchical_cluster' n'existe pas, on applique le clustering
if 'hierarchical_cluster' not in df_clustering_without_duplicates.columns:
    hierarchical_cluster = AgglomerativeClustering(n_clusters=20)  # Ajustez le nombre de clusters
    df_clustering_without_duplicates['hierarchical_cluster'] = hierarchical_cluster.fit_predict(df_clustering_without_duplicates[['lat', 'long']])

# Palette de couleurs
palette = sns.color_palette("hsv", n_colors=df_clustering_without_duplicates['hierarchical_cluster'].nunique())
colors = [mcolors.rgb2hex(color) for color in palette]

# Calculer les centroids manuellement pour chaque cluster
centroids = df_clustering_without_duplicates.groupby('hierarchical_cluster')[['lat', 'long']].mean().values

# Centrer la carte
m = folium.Map(location=[df_clustering_without_duplicates['lat'].mean(), df_clustering_without_duplicates['long'].mean()], zoom_start=12)

# Filtrer les points basés sur la distance aux centroids
filtered_points = df_clustering_without_duplicates[df_clustering_without_duplicates.apply(
    lambda row: any(geodesic((row['lat'], row['long']), centroid).m <= 200 for centroid in centroids), axis=1)]

# Ajouter des points filtrés à la carte
for i, row in filtered_points.iterrows():
    point = (row['lat'], row['long'])
    cluster_id = int(row['hierarchical_cluster'])  # Convertir en entier
    folium.CircleMarker(
        location=point,
        radius=1,
        color=colors[cluster_id],  # Utiliser 'hierarchical_cluster' pour la couleur
        fill=True,
        fill_color=colors[cluster_id]
    ).add_to(m)

# Dessiner les Convex Hulls et les centroids
for cluster_id in range(df_clustering_without_duplicates['hierarchical_cluster'].nunique()):
    cluster_points = filtered_points[filtered_points['hierarchical_cluster'] == cluster_id][['lat', 'long']].values
    if len(cluster_points) >= 3:  # ConvexHull nécessite au moins 3 points
        try:
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except Exception as e:
            print(f"Erreur avec le cluster {cluster_id}: {e}")

    centroid = centroids[cluster_id]
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Centroid {cluster_id}',
    ).add_to(m)

# Sauvegarder la carte
m.save("../html/hierarchical.html")


## DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
def plot_k_distance_graph(X, k):
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, _ = neigh.kneighbors(X)
    distances = np.sort(distances[:, k-1])
    plt.figure(figsize=(10, 10))
    plt.plot(distances)
    plt.xlabel('Points')
    plt.ylabel(f'{k}-th nearest neighbor distance')
    plt.title('K-distance Graph')

    y_min, y_max = 0, 0.1
    y_ticks = np.arange(y_min, y_max, 0.01)  # Création des ticks de 0.001 en 0.001
    plt.yticks(y_ticks)
    plt.grid(axis='y', which='both', linestyle='--', linewidth=0.5)
    plt.show()
# Plot k-distance graph
plot_k_distance_graph(scaled_data_df, k=5)

In [ ]:


dbscan = DBSCAN(eps=0.0058, min_samples=56)  

dbscan.fit(scaled_data_df)
data_cleaned['cluster_dbscan'] = dbscan.labels_
print(data_cleaned['cluster_dbscan'].value_counts())

In [ ]:


# Créer une carte centrée sur la moyenne des coordonnées
m = folium.Map(location=[data_cleaned['lat'].mean(), data_cleaned['long'].mean()], zoom_start=12)

# Filtrer les clusters valides (exclure le bruit: cluster -1)
valid_clusters = data_cleaned['cluster_dbscan'][data_cleaned['cluster_dbscan'] != -1].unique()
palette = sns.color_palette("hsv", n_colors=len(valid_clusters))
colors = {cluster: mcolors.rgb2hex(color) for cluster, color in zip(valid_clusters, palette)}

# Sous-échantillonnage
sampled_data = data_cleaned.sample(n=min(20000, len(data_cleaned)), random_state=1)

# Ajouter les points de clusters à la carte
for i, row in sampled_data.iterrows():
    cluster_id = row['cluster_dbscan']
    if cluster_id == -1:  # Ignorer le bruit
        continue
    folium.CircleMarker(
        location=(row['lat'], row['long']),
        radius=1,
        color=colors[cluster_id],
        fill=True,
        fill_color=colors[cluster_id]
    ).add_to(m)

# Calculer les barycentres des clusters pour les représenter comme "centroïdes"
for cluster_id in valid_clusters:
    cluster_points = data_cleaned[data_cleaned['cluster_dbscan'] == cluster_id][['lat', 'long']].values
    
    if len(cluster_points) == 0:
        continue
    
    # Barycentre du cluster
    centroid = cluster_points.mean(axis=0)
    
    # Ajouter un marqueur pour le barycentre
    folium.Marker(
        location=[centroid[0], centroid[1]],
        popup=f'Cluster {cluster_id}',
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

    # Dessiner l'enveloppe convexe si au moins 3 points
    if len(cluster_points) >= 3:
        try:
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            folium.Polygon(
                locations=hull_points,
                color=colors[cluster_id],
                fill=True,
                fill_opacity=0.2
            ).add_to(m)
        except QhullError:
            print(f"Skipping cluster {cluster_id} due to QhullError")

# Sauvegarder la carte
m.save("../html/dbscan_map.html")

In [153]:

from pyproj import Proj, Transformer

dbscan = DBSCAN(eps=0.02, min_samples=4)  
dbscan.fit(scaled_data_df)
data_cleaned['cluster_dbscan'] = dbscan.labels_



# Définir la projection UTM pour la zone (ex: Lyon = UTM zone 31T)
proj = Proj(proj="utm", zone=31, ellps="WGS84", datum="WGS84")
transformer = Transformer.from_proj(Proj("epsg:4326"), proj)

# Appliquer la transformation
scaled_data_df['x'], scaled_data_df['y'] = transformer.transform(df_clustering['lat'], df_clustering['long'])


# Afficher le résultat


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
def plot_k_distance_graph(X, k):
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, _ = neigh.kneighbors(X)
    distances = np.sort(distances[:, k-1])
    plt.figure(figsize=(10, 20))
    plt.plot(distances)
    plt.xlabel('Points')
    plt.ylabel(f'{k}-th nearest neighbor distance')
    plt.title('K-distance Graph')

    y_min, y_max = 0, 100
    y_ticks = np.arange(y_min, y_max, 10)  # Création des ticks de 0.001 en 0.001
    plt.yticks(y_ticks)
    plt.grid(axis='y', which='both', linestyle='--', linewidth=0.5)
    plt.show()
# Plot k-distance graph
plot_k_distance_graph(scaled_data_df, k=5)